In [33]:
from pdfparsing import read_text_file, extract_table
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill, Protection
import numpy as np

In [3]:
df = {}
file_names, desired_table_name_list = read_text_file()
for file_name in file_names:
    df[file_name] = extract_table(file_name, desired_table_name_list)

c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [22]:
for table_name in desired_table_name_list:
    max_depth = 0
    title_list = []
    for file_name in file_names:
        if isinstance(df[file_name][table_name], pd.DataFrame):
            depth = df[file_name][table_name]["과목"].apply(lambda x: x.count("_")).max()
            if max_depth < depth:
                max_depth = depth
            title_list.extend(df[file_name][table_name]["과목"].values)
    max_depth += 1                  # _가 하나도 없으면 depth를 1로 정하려고.
    title_list = list(set(title_list))
    sorted_title_list = sorted(title_list, key=lambda x: tuple(x.split("_") + [""] * (max_depth - len(x.split("_")))))
    print(sorted_title_list)
    
    

['부채', '부채_부채총계', '부채_비유동부채', '부채_비유동부채_기타 비유동 부채', '부채_비유동부채_기타금융부채', '부채_비유동부채_기타부채', '부채_비유동부채_리스부채', '부채_비유동부채_비유동금융부채', '부채_비유동부채_비유동비금융부채', '부채_비유동부채_비유동충당부채', '부채_비유동부채_사채', '부채_비유동부채_순확정급여부채', '부채_비유동부채_이연법인세부채', '부채_비유동부채_장기매입채무 및 기타비유동채무', '부채_비유동부채_장기미지급금', '부채_비유동부채_장기성미지급금', '부채_비유동부채_장기차입금', '부채_비유동부채_장기충당부채', '부채_비유동부채_종업원급여채무', '부채_비유동부채_충당부채', '부채_유동부채', '부채_유동부채_기타 유동부채', '부채_유동부채_기타금융부채', '부채_유동부채_기타부채', '부채_유동부채_단기차입금', '부채_유동부채_당기법인세부채', '부채_유동부채_리스부채', '부채_유동부채_매각예정분류부채', '부채_유동부채_매각예정비유동부채', '부채_유동부채_매입채무', '부채_유동부채_매입채무및기타채무', '부채_유동부채_미지급금', '부채_유동부채_미지급비용', '부채_유동부채_선수금', '부채_유동부채_예수금', '부채_유동부채_유동금융부채', '부채_유동부채_유동비금융부채', '부채_유동부채_유동성장기부채', '부채_유동부채_유동충당부채', '부채_유동부채_충당부채', '부채와자본총계', '자본', '자본_비지배지분', '자본_자본총계', '자본_지배기업 소유주지분', '자본_지배기업 소유주지분_기타자본항목', '자본_지배기업 소유주지분_이익잉여금', '자본_지배기업 소유주지분_자본금', '자본_지배기업 소유주지분_자본금_보통주자본금', '자본_지배기업 소유주지분_자본금_우선주자본금', '자본_지배기업 소유주지분_주식발행초과금', '자본_지배기업소유주지분', '자본_지배기업소유주지분_기타자본항목', '자본_지배기업소유주지분_기타포괄손익누계액', '자본_지배기업소유주지분_이익잉여금

In [47]:
# 이거 좀 고치자...
def process_financial_statements_by_statement(df):
    result = {}
    statements = []
    companies = df.keys()
    for company in companies:
        statements.extend(df[company].keys())
    statements = list(set(statements))
    
    for statement in statements:
        for cols in [[0, 1], [0, 2], [0, 3]]:
            temp_frames = []
            valid_companies = []
            
            for company in companies:
                if company.find(".") != -1:             # 확장자 부분 자르기
                    company = company[:company.find(".")]
                company_name = company.lower()
                
                # 데이터프레임이 -1 이면 무시
                if company_name in df and statement in df[company_name]:
                    if isinstance(df[company_name][statement], pd.DataFrame):
                        temp = df[company_name][statement].iloc[:, cols].T
                        temp.columns = temp.iloc[0, :]
                        temp = temp.drop(temp.index[0])
                        
                        temp_frames.append(temp)
                        company = company.capitalize()      # 맨 앞글자만 대문자로
                        valid_companies.append(company)
            
            if temp_frames:
                combined_result = pd.concat(temp_frames, keys=valid_companies)
                combined_result.index = valid_companies
                
                total = combined_result.sum(min_count=1)
                combined_result.loc["Sum"] = total
                
                avg = combined_result.iloc[:-1].mean()      # 합계까지 더해 평균을 더하는 문제 방지
                combined_result.loc["Avg"] = avg
                combined_result = combined_result.replace(np.nan, "-")
                combined_result = combined_result.sort_index(axis=1, ascending=True)

                year = temp_frames[0].index[0]
                
                if statement not in result:
                    result[statement] = {}
                
                result[statement][year] = combined_result
    
    return result

def table_to_json(df):
    result = {}
    for year in df:
        df_of_this_year = df[year]
        for title in df_of_this_year:
            column_list = df[year][title].columns
            for column in column_list:
                value = df[year][title][column]
                ancestors = [year] + column.split("_")
                current = result
                for key in ancestors:
                    if key not in current:
                        current[key] = {}
                    current = current[key]
                current["Value"] = dict(value)
    return result

In [51]:
df_csv = process_financial_statements_by_statement(df)

In [52]:
df_csv

{'연결 재무상태표': {'2023': 과목      부채           부채_부채총계         부채_비유동부채 부채_비유동부채_기타 비유동 부채  \
  Samsung  -          92228115         16508663            2802356   
  Hyundai  -         180653915        107291812                  -   
  Kepco    -         202450215        141201794                  -   
  Sum      -         475332245        265002269            2802356   
  Avg      -  158444081.666667  88334089.666667          2802356.0   
  
  과목      부채_비유동부채_기타금융부채 부채_비유동부채_기타부채 부채_비유동부채_리스부채 부채_비유동부채_비유동금융부채  \
  Samsung               -             -             -                -   
  Hyundai          176399       5212012        834052                -   
  Kepco                 -             -             -         92944338   
  Sum              176399       5212012        834052         92944338   
  Avg            176399.0     5212012.0      834052.0       92944338.0   
  
  과목      부채_비유동부채_비유동비금융부채 부채_비유동부채_비유동충당부채  ... 자산_유동자산_매각예정자산 자산_유동자산_매출채권  \
  Samsung                 -  

In [ ]:
def table_to_json(df):
    result = {}
    for year in df:
        df_of_this_year = df[year]
        for title in df_of_this_year:
            column_list = df[year][title].columns
            for column in column_list:
                value = df[year][title][column]
                ancestors = [year] + column.split("_")
                current = result
                for key in ancestors:
                    if key not in current:
                        current[key] = {}
                    current = current[key]
                current["Value"] = dict(value)
    return result